In [13]:
# Natural Language Toolkit: Eliza
#
# Copyright (C) 2001-2023 NLTK Project
# Authors: Steven Bird <stevenbird1@gmail.com>
#          Edward Loper <edloper@gmail.com>
# URL: <https://www.nltk.org/>
# For license information, see LICENSE.TXT

# importing the NLTK 
import nltk

# The usage for the above lines is to include/read certain words that will be...
# used to fill in the gaps for the automated responses for the chatbot

from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

# importing the NLTK package that allows word tokenizing
from nltk import word_tokenize
# importing the NLTK package that allows stemming words
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
# importing the NLTK package that deals with regular expression stemming
from nltk.stem import RegexpStemmer

from nltk.chat.util import Chat, reflections

# a table of response pairs, where each pair consists of a
# regular expression, and a list of possible responses,
# with group-macros labelled as %1, %2.

pairs = (
    (
        r"I need (.*)",
        (
            "Why do you need %1?",
            "Would it really help you to get %1?",
            "Are you sure you need %1?",
        ),
    ),
    (
        r"Why don\'t you (.*)",
        (
            "Do you really think I don't %1?",
            "Perhaps eventually I will %1.",
            "Do you really want me to %1?",
        ),
    ),
    (
        r"Why can\'t I (.*)",
        (
            "Do you think you should be able to %1?",
            "If you could %1, what would you do?",
            "I don't know -- why can't you %1?",
            "Have you really tried?",
        ),
    ),
    (
        r"I can\'t (.*)",
        (
            "How do you know you can't %1?",
            "Perhaps you could %1 if you tried.",
            "What would it take for you to %1?",
        ),
    ),
    (

        r"I am (.*)",
        (
            "Did you come to me because you are %1?",
            "How long have you been %1?",
            "How do you feel about being %1?",
        ),
    ),
    (

        r"I\'m (.*)",
        (
            "How does being %1 make you feel?",
            "Do you enjoy being %1?",
            "Why do you tell me you're %1?",
            "Why do you think you're %1?",
        ),
    ),
    (

        r"Are you (.*)",
        (
            "Why does it matter whether I am %1?",
            "Would you prefer it if I were not %1?",
            "Perhaps you believe I am %1.",
            "I may be %1 -- what do you think?",
        ),
    ),
    (

        r"What (.*)",
        (
            "Why do you ask?",
            "How would an answer to that help you?",
            "What do you think?",
        ),
    ),
    (

        r"How (.*)",
        (
            "How do you suppose?",
            "Perhaps you can answer your own question.",
            "What is it you're really asking?",
        ),
    ),
    (

        r"Because (.*)",
        (
            "Is that the real reason?",
            "What other reasons come to mind?",
            "Does that reason apply to anything else?",
            "If %1, what else must be true?",
        ),
    ),
    (

        r"(.*) sorry (.*)",
        (
            "There are many times when no apology is needed.",
            "What feelings do you have when you apologize?",
        ),
    ),
    (
        r"Hello(.*)",
        (
            "Hello... I'm glad you could drop by today.",
            "Hi there... how are you today?",
            "Hello, how are you feeling today?",
        ),
    ),
    (

        r"I think (.*)",
        ("Do you doubt %1?", "Do you really think so?", "But you're not sure %1?"),
    ),
    (
        r"(.*) friend (.*)",
        (
            "Tell me more about your friends.",
            "When you think of a friend, what comes to mind?",
            "Why don't you tell me about a childhood friend?",
        ),
    ),
    (r"Yes", ("You seem quite sure.", "OK, but can you elaborate a bit?")),
    (
        r"(.*) computer(.*)",
        (
            "Are you really talking about me?",
            "Does it seem strange to talk to a computer?",
            "How do computers make you feel?",
            "Do you feel threatened by computers?",
        ),
    ),
    (
        r"Is it (.*)",
        (
            "Do you think it is %1?",
            "Perhaps it's %1 -- what do you think?",
            "If it were %1, what would you do?",
            "It could well be that %1.",
        ),
    ),
    (
        r"It is (.*)",
        (
            "You seem very certain.",
            "If I told you that it probably isn't %1, what would you feel?",
        ),
    ),
    (
        r"Can you (.*)",
        (
            "What makes you think I can't %1?",
            "If I could %1, then what?",
            "Why do you ask if I can %1?",
        ),
    ),
    (
        r"Can I (.*)",
        (
            "Perhaps you don't want to %1.",
            "Do you want to be able to %1?",
            "If you could %1, would you?",
        ),
    ),
    (
        r"You are (.*)",
        (
            "Why do you think I am %1?",
            "Does it please you to think that I'm %1?",
            "Perhaps you would like me to be %1.",
            "Perhaps you're really talking about yourself?",
        ),
    ),
    (
        r"You\'re (.*)",
        (
            "Why do you say I am %1?",
            "Why do you think I am %1?",
            "Are we talking about you, or me?",
        ),
    ),
    (
        r"I don\'t (.*)",
        ("Don't you really %1?", "Why don't you %1?", "Do you want to %1?"),
    ),
    (
        r"I feel (.*)",
        (
            "Good, tell me more about these feelings.",
            "Do you often feel %1?",
            "When do you usually feel %1?",
            "When you feel %1, what do you do?",
        ),
    ),
    (
        r"I have (.*)",
        (
            "Why do you tell me that you've %1?",
            "Have you really %1?",
            "Now that you have %1, what will you do next?",
        ),
    ),
    (
        r"I would (.*)",
        (
            "Could you explain why you would %1?",
            "Why would you %1?",
            "Who else knows that you would %1?",
        ),
    ),
    (
        r"Is there (.*)",
        (
            "Do you think there is %1?",
            "It's likely that there is %1.",
            "Would you like there to be %1?",
        ),
    ),
    (
        r"My (.*)",
        (
            "I see, your %1.",
            "Why do you say that your %1?",
            "When your %1, how do you feel?",
        ),
    ),
    (
        r"You (.*)",
        (
            "We should be discussing you, not me.",
            "Why do you say that about me?",
            "Why do you care whether I %1?",
        ),
    ),
    (r"Why (.*)", ("Why don't you tell me the reason why %1?", "Why do you think %1?")),
    (
        r"I want (.*)",
        (
            "What would it mean to you if you got %1?",
            "Why do you want %1?",
            "What would you do if you got %1?",
            "If you got %1, then what would you do?",
        ),
    ),
    (
        r"(.*) mother(.*)",
        (
            "Tell me more about your mother.",
            "What was your relationship with your mother like?",
            "How do you feel about your mother?",
            "How does this relate to your feelings today?",
            "Good family relations are important.",
        ),
    ),
    (
        r"(.*) father(.*)",
        (
            "Tell me more about your father.",
            "How did your father make you feel?",
            "How do you feel about your father?",
            "Does your relationship with your father relate to your feelings today?",
            "Do you have trouble showing affection with your family?",
        ),
    ),
    (
        r"(.*) child(.*)",
        (
            "Did you have close friends as a child?",
            "What is your favorite childhood memory?",
            "Do you remember any dreams or nightmares from childhood?",
            "Did the other children sometimes tease you?",
            "How do you think your childhood experiences relate to your feelings today?",
        ),
    ),
    (
        r"(.*)\?",
        (
            "Why do you ask that?",
            "Please consider whether you can answer your own question.",
            "Perhaps the answer lies within yourself?",
            "Why don't you tell me?",
        ),
    ),
    (
        r"(.*) Finals (.*)",
        (
            "I hear you. Finals week is tough for many. Do you want to tell me more?",
            "You are not the only person who feels this way. Do not give up!",
        )
    ),
    (
        r"(.*) college (.*)",
        (
            "It is amazing that you are in college!",
            "How do you feel about going to college?",
        )
    ),
    (
        r"(.*) difficult (.*)",
        (
            "You are not the only person who finds it difficult.",
            "How does that make you feel?",
        )
    ),
    (
        r"(.*) listen (.*)",
        (
            "I am here to listen to you!",
            "What would you like to tell me?",
        )
    ),
    (
        r"(.*) fail (.*)",
        (
            "You are not going to fail.",
            "What makes you think that you are going to fail?",
        )
    ),
    (
        r"quit",
        (
            "Thank you for talking with me.",
            "Good-bye.",
            "Thank you, that will be $150.  Have a good day!",
        ),
    ),
    (
        r"(.*)",
        (
            "Please tell me more.",
            "Let's change focus a bit... Tell me about your family.",
            "Can you elaborate on that?",
            "Why do you say that %1?",
            "I see.",
            "Very interesting.",
            "%1.",
            "I see.  And what does that tell you?",
            "How does that make you feel?",
            "How do you feel when you say that?",
        ),
    ),
)


# The call to the series of responses/dialogues registered for the chatbot using the NLTK database
eliza_chatbot = Chat(pairs,reflections)


# Function to start the display the rules/guidelines for beginning the conversation for the user with the chatbox.
def eliza_chat():
    
    # tokenizing and printed the original vs the tokenized strings
    tokenizerTest = '''This is a test string to see if tokenizing works.'''
    print("\nBefore tokenizing: ", tokenizerTest)
    # calling upon the word_tokenizer() function to do the tokenizing
    print("After tokenizing: ", word_tokenize(tokenizerTest))
    
    # stemming and pritning the original words, and stemmed words after.
    stemTest = ["code", "codes", "coding"]
    print("\nBefore stemming, the words are: ")
    # iterating through the array and printing each word
    for word1 in stemTest:
        print(word1)
    # calling upon the PorterStemmer() function to stem the words
    ps = PorterStemmer()
    print("\nAfter stemming, the words are: ")
    # once again iterating through the array, and printing each word, and its stemmed version
    for word2 in stemTest:
        print(word2, " : ", ps.stem(word2))
    
    # using regular expressions to stem words
    regexTest = ["mass", "doable", "working"]
    print("\nBefore using Regular Expression, the words are:\n")
    for word3 in regexTest:
        print(word3)
    # using the RegexpStemmer() function to state that words ending in "ing", "s", "e", "able" will be stemmed as long as the word is at least 4 characters long
    # found this function from the NLTK website : https://www.nltk.org/api/nltk.stem.regexp.html
    regex = RegexpStemmer('ing$|s$|e$|able$', min = 4)
    print("\nAfter using Regular Expression, the words are:\n")
    for word4 in regexTest:
        print(regex.stem(word4))
    
    # I was successful in figuring out which functions to use to tokenize and stem words in a sentence.
    # However, I was unable to use those functions within the chatbot to make it so that
    # every user response will get tokenized. All of my approaches to modify the chatbot responses
    # ended in errors. Had I started the assignment earlier, I would have been able to accommodate for
    # this issue. However, due to my poor planning, I ended up not being able to submit a program that truly
    # and perfectly satisfies the requirements of the assignment. I will try my best to do better
    # next time.
    
    print("\nTherapist\n---------")
    print("Talk to the program by typing in plain English, using normal upper-")
    print('and lower-case letters and punctuation.  Enter "quit" when done.')
    print("=" * 72)
    print("Hello.  How are you feeling today?")
    

    eliza_chatbot.converse()

# This function will call upon the "eliza_chat()" function which will invoke the chatbot,
def demo():
    eliza_chat()


if __name__ == "__main__":
    demo()

[nltk_data] Error loading popular: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>

Before tokenizing:  This is a test string to see if tokenizing works.
After tokenizing:  ['This', 'is', 'a', 'test', 'string', 'to', 'see', 'if', 'tokenizing', 'works', '.']

Before stemming, the words are: 
code
codes
coding

After stemming, the words are: 
code  :  code
codes  :  code
coding  :  code

Before using Regular Expression, the words are:

mass
doable
working

After using Regular Expression, the words are:

mas
do
work

Therapist
---------
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
Hello.  How are you feeling today?


> I am okay.


How do you feel about being okay?


> I am not against that


How long have you been not against that?


...

Why don't you tell me the reason why not?


> I don't know!


Do you want to know?


> quit


Thank you for talking with me.
